In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab


In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.6)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%%time
!rm -r frames/*
!mkdir frames/

In [ ]:
!cd frames

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:

!pwd

In [ ]:
video = "/content/sample.mp4"

cap = cv2.VideoCapture(video)
cnt = 0

if(cap.isOpened() == False):
  print("Error opening video stream or file")

ret,first_frame = cap.read()

while(cap.isOpened()):
  ret,frame = cap.read()
  if(ret == True):
    cv2.imwrite('frames/'+str(cnt)+'.png',frame)
    cnt = cnt + 1
    if(cnt == 750):
      break
  else:
    break

In [ ]:
FPS = cap.get(cv2.CAP_PROP_FPS)
print(FPS)

In [ ]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml")
predictor = DefaultPredictor(cfg)

In [ ]:
img = cv2.imread("frames/30.png")

outputs = predictor(img)

In [ ]:
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
classes = outputs['instances'].pred_classes.cpu().numpy()
print(classes)

In [ ]:
bbox = outputs['instances'].pred_boxes.tensor.cpu().numpy()
print(bbox)

In [ ]:
ind = np.where(classes == 0)[0]

person = bbox[ind]#bbox of only people

num = len(person)#total no of people in a single frame

In [ ]:
#bbox format

x1,y1,x2,y2 = person[0]
print(x1,y1,x2,y2)

In [ ]:
img = cv2.imread('frames/30.png')
_=cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),2)

plt.figure(figsize=(20,10))
plt.imshow(img)

In [ ]:
#compute center

x_center = int((x1+x2)/2)
y_center = int(y2)

center = (x_center, y_center)

_ = cv2.circle(img, center, 5, (255, 0, 0), -1)
plt.figure(figsize = (20,10))
plt.imshow(img)

In [ ]:
def mid_point(img,person,idx):
  x1,y1,x2,y2 = person[idx]
  _=cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)

  x_mid = int((x1+x2)/2)
  y_mid = int(y2)
  mid = (x_mid, y_mid)

  _=cv2.circle(img,mid,5,(0,255,0),-1)
  cv2.putText(img, str(idx), mid, cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

  return mid

In [ ]:
midpoints = [mid_point(img,person,i) for i in range(len(person))]

plt.figure(figsize=(20,10))
plt.imshow(img)

In [ ]:
%%time
from scipy.spatial import distance
def compute_distance(midpoints,num):
  dist = np.zeros((num,num))
  for i in range(num):
    for j in range(i+1, num):
      if i!=j:
        dst = distance.euclidean(midpoints[i], midpoints[j])
        dist[i][j] = dst
  return dist

In [ ]:
dist = compute_distance(midpoints,num)

In [ ]:
def find_closest(dist,num,thresh):
  p1 = []
  p2 = []
  d = []
  for i in range(num):
    for j in range(i,num):
      if((i!=j) & (dist[i][j] <= thresh)):
        p1.append(i)
        p2.append(j)
        d.append(dist[i][j])
  return p1,p2,d

In [ ]:
import pandas as pd

thresh = 100
p1,p2,d = find_closest(dist,num,thresh)
df = pd.DataFrame({"p1":p1,"p2":p2,"dist":d})
df

In [ ]:
def change_2_red(img,person,p1,p2):
  risky = np.unique(p1+p2)
  for i in risky:
    x1,y1,x2,y2 = person[i]
    _ = cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),2)
  return img

In [ ]:
img = change_2_red(img,person,p1,p2)

plt.figure(figsize = (20,10))
plt.imshow(img)

In [ ]:
import os
import re

names=os.listdir('frames/')
names.sort(key=lambda f: int(re.sub('\D', '', f)))

In [ ]:
def find_closest_people(name,thresh):

  img = cv2.imread('frames/'+name)
  outputs = predictor(img)
  classes=outputs['instances'].pred_classes.cpu().numpy()
  bbox=outputs['instances'].pred_boxes.tensor.cpu().numpy()
  ind = np.where(classes==0)[0]
  person=bbox[ind]
  midpoints = [mid_point(img,person,i) for i in range(len(person))]
  num = len(midpoints)
  dist= compute_distance(midpoints,num)
  p1,p2,d=find_closest(dist,num,thresh)
  img = change_2_red(img,person,p1,p2)
  cv2.imwrite('frames/'+name,img)
  return 0

In [ ]:
from tqdm import tqdm
thresh=100
_ = [find_closest_people(names[i],thresh) for i in tqdm(range(len(names))) ]

In [ ]:
%%time
frames = os.listdir('frames/')
frames.sort(key=lambda f: int(re.sub('\D', '', f)))

frame_array=[]

for i in range(len(frames)):
    
    #reading each files
    img = cv2.imread('frames/'+frames[i])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)

out = cv2.VideoWriter('sample_output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
 
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()